<a href="https://colab.research.google.com/github/VesalAhsani/Driver-behavior-detection/blob/main/Mixed_Dataset_100Driver_SynDD2_Grayscale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Check if the dataset zip file exists in Google Drive
import os
dataset_path = '/content/drive/MyDrive/SynDD2_mixed_224.zip'  # Adjust the path if needed
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"The dataset file at {dataset_path} was not found!")
print("Dataset zip file exists.")

Dataset zip file exists.


In [3]:
# Unzipping the dataset
import zipfile
extract_path = '/content/SynDD2_mixed_224'
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Dataset unzipped successfully!")

Dataset unzipped successfully!


In [5]:
# Check the actual folder structure after unzipping
dataset_actual_path = os.path.join(extract_path, "SynDD2_mixed_224")  # Adjust to the nested folder
if not os.path.exists(os.path.join(dataset_actual_path, "train")):
    raise FileNotFoundError("Train folder not found in the dataset path!")

In [6]:
# Install the YOLOv8 library
!pip install ultralytics --upgrade

# Import the YOLO library
from ultralytics import YOLO
print("Ultralytics library imported successfully!")

# Verify the dataset structure
from ultralytics.data.utils import check_cls_dataset
dataset_info = check_cls_dataset(dataset_actual_path)
print(f"Dataset verified: {dataset_info}")

Ultralytics library imported successfully!
train: /content/SynDD2_mixed_224/SynDD2_mixed_224/train... found 115001 images in 11 classes ✅ 
val: /content/SynDD2_mixed_224/SynDD2_mixed_224/val... found 14375 images in 11 classes ✅ 
test: /content/SynDD2_mixed_224/SynDD2_mixed_224/test... found 14376 images in 11 classes ✅ 
Dataset verified: {'train': PosixPath('/content/SynDD2_mixed_224/SynDD2_mixed_224/train'), 'val': PosixPath('/content/SynDD2_mixed_224/SynDD2_mixed_224/val'), 'test': PosixPath('/content/SynDD2_mixed_224/SynDD2_mixed_224/test'), 'nc': 11, 'names': {0: 'Drinking', 1: 'Eating', 2: 'Normal', 3: 'Phone_Call_(Left)', 4: 'Phone_Call_(Right)', 5: 'Reaching_Behind', 6: 'Sleep', 7: 'Smoking', 8: 'Text_(Left)', 9: 'Text_(Right)', 10: 'Yawning'}}


In [7]:
# Count the number of training and validation images
import glob
train_images = glob.glob(os.path.join(dataset_actual_path, "train", "*", "*.jpg"))
val_images = glob.glob(os.path.join(dataset_actual_path, "val", "*", "*.jpg"))

if not train_images or not val_images:
    raise ValueError("Training or validation images were not found. Check dataset paths.")
print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(val_images)}")

Number of training images: 115001
Number of validation images: 14375


In [8]:
# Define paths
DATASET_PATH = dataset_actual_path  # Path to the unzipped dataset
OUTPUT_PATH = '/content/drive/MyDrive/yolov8_cls_results'

# Initialize the YOLOv8 model
model = YOLO('yolov8s-cls.pt')  # Using yolov8s-cls as a starting point
print("YOLOv8 model initialized with pretrained weights.")

# Train the model
model.train(
    data=DATASET_PATH,  # Path to dataset
    epochs=50,          # Adjust based on your needs
    batch=32,           # Adjust according to GPU memory
    imgsz=224,          # Image size, as per your dataset
    project=OUTPUT_PATH,  # Save results to your Google Drive
    name='yolov8_cls_224',  # Run name
    pretrained=True,     # Start from pretrained weights
    optimizer='Adam',    # Optimizer
    lr0=0.001            # Initial learning rate
)
print("Training completed!")

# Evaluate the model
results = model.val()
print("Validation completed!")

# Save the best weights
best_weights_path = f"{OUTPUT_PATH}/yolov8_cls_224/weights/best.pt"
os.makedirs(f"{OUTPUT_PATH}/yolov8_cls_224/weights", exist_ok=True)
!cp /content/runs/classify/yolov8_cls_224/weights/best.pt {best_weights_path}
print(f"Best weights saved to {best_weights_path}")

100%|██████████| 12.3M/12.3M [00:00<00:00, 42.2MB/s]


YOLOv8 model initialized with pretrained weights.
Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=/content/SynDD2_mixed_224/SynDD2_mixed_224, epochs=50, time=None, patience=100, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8_cls_results, name=yolov8_cls_224, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=F

100%|██████████| 5.35M/5.35M [00:00<00:00, 342MB/s]


AMP: checks passed ✅


train: Scanning /content/SynDD2_mixed_224/SynDD2_mixed_224/train... 115001 images, 0 corrupt: 100%|██████████| 115001/115001 [00:35<00:00, 3225.53it/s]


...


50 epochs completed in 2.042 hours.
Optimizer stripped from /content/drive/MyDrive/yolov8_cls_results/yolov8_cls_224/weights/last.pt, 10.3MB
Optimizer stripped from /content/drive/MyDrive/yolov8_cls_results/yolov8_cls_224/weights/best.pt, 10.3MB

Validating /content/drive/MyDrive/yolov8_cls_results/yolov8_cls_224/weights/best.pt...
Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv8s-cls summary (fused): 73 layers, 5,089,291 parameters, 0 gradients, 12.5 GFLOPs
train: /content/SynDD2_mixed_224/SynDD2_mixed_224/train... found 115001 images in 11 classes ✅ 
val: /content/SynDD2_mixed_224/SynDD2_mixed_224/val... found 14375 images in 11 classes ✅ 
test: /content/SynDD2_mixed_224/SynDD2_mixed_224/test... found 14376 images in 11 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 225/225 [00:06<00:00, 36.36it/s]


                   all      0.968          1
Speed: 0.1ms preprocess, 0.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8_cls_results/yolov8_cls_224
Training completed!
Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv8s-cls summary (fused): 73 layers, 5,089,291 parameters, 0 gradients, 12.5 GFLOPs
train: /content/SynDD2_mixed_224/SynDD2_mixed_224/train... found 115001 images in 11 classes ✅ 
val: /content/SynDD2_mixed_224/SynDD2_mixed_224/val... found 14375 images in 11 classes ✅ 
test: /content/SynDD2_mixed_224/SynDD2_mixed_224/test... found 14376 images in 11 classes ✅ 


val: Scanning /content/SynDD2_mixed_224/SynDD2_mixed_224/val... 14375 images, 0 corrupt: 100%|██████████| 14375/14375 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 450/450 [00:06<00:00, 71.44it/s]


                   all      0.968          1
Speed: 0.1ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8_cls_results/yolov8_cls_2242
Validation completed!
cp: cannot stat '/content/runs/classify/yolov8_cls_224/weights/best.pt': No such file or directory
Best weights saved to /content/drive/MyDrive/yolov8_cls_results/yolov8_cls_224/weights/best.pt
